# Assignment 3: Linear Regression (100 points)

Only use the already imported library `numpy` and the Python standard library. Make sure that the datasets `dataLinReg2D.txt`, `dataQuadReg2D.txt` and `dataQuadReg2D_noisy.txt` are in the same directory as the notebook.


==> *Write*
* *names* 
* *matr. nr.* 
* *study program*
* *B.Sc./M.Sc.*

*of all assignment group participants here. (double klick here to edit)*

## Task 1: Linear Regression - theory (10 points)
1) *(4 points)* Linear regression can have nonlinear input features, why is it still called linear regression? In what sense is it linear?

==> *Write your response here.* (double klick here to edit)

2) *(3 points)* For calculating optimal parameters $\hat{\beta}$ the inverse of $X^{\top}X$ has to be calculated. When would this matrix be singular?

==> *Write your response here.* (double klick here to edit)

3) *(3 points)* Why does the optimization problem for $L_1$-regularization (Lasso) not have a closed form solution?

==> *Write your response here.* (double klick here to edit)

## Task 2: Ridge Regression for Polynomial 2D Functions (70 points)
Each line in the data sets consists of a data entry `(x,y)` with a 2D point `x` and a 1D function output `y`.

In [2]:
# Load required packages and datasets. Do not modify.
import numpy as np


def load_dataset(path):
    data = np.loadtxt(path)
    X, y = data[:, :2], data[:, 2]
    
    return X, y
    

X_lin, y_lin = load_dataset("dataLinReg2D.txt")
X_quad, y_quad = load_dataset("dataQuadReg2D.txt")
X_noise, y_noise = load_dataset("dataQuadReg2D_noisy.txt")

1) *(5 points)* Draw random samples from the dataset for training, use 70% of the data. Do not modify the interface of the function. Do this for `X_quad` and `X_noise`.

In [13]:
def train_split(X, y, test_split = 0.7):
    """
    Returns X_train, y_train
        where X_train are random samples of X and y_train are the corresponding true values.
        test_split represents the persentage of how many training samples are drawn from X.
    """
    # Implement your solution here.
    np.random.seed(2020)  # Ensure that the random split always returns the same result.
    
    threshold = int(test_split*X.shape[0])
    rnd_idx = np.random.permutation(X.shape[0])
    
    X_train = X[rnd_idx[:threshold]]
#    X_test = X[rnd_idx[threshold:]]
    y_train = y[rnd_idx[:threshold]]
#    y_test = y[rnd_idx[threshold:]]
    return X_train, y_train

X_quad_train, y_quad_train = train_split(X_quad, y_quad)
X_noise_train, y_noise_train = train_split(X_noise, y_noise)
print(len(X_quad_train), y_quad)
print(len(X_noise_train), len(y_noise_train))

70 [ 1.45131  2.10077  5.2415  14.2015   6.28687  2.63999  1.65735  1.84973
  1.5936   5.93914 17.3139   2.79152  1.28165  4.97582  2.06822  4.29598
  2.12421  4.51654  2.06546  1.49823  5.86448  1.5392   3.82217  5.08817
  9.51408  1.30278  5.44264  2.15566  1.59536  6.75806 13.5923  23.663
  2.43735  6.75188  1.70615  4.46588  1.80535  2.59595  5.02799  2.58967
  4.06867  3.45738 15.1917   6.39383  4.25406  3.31947  5.01941  4.23633
  2.2513   1.54746  3.86675  1.88672  1.5219   7.55069  4.67213  1.75698
  4.43193  3.25097  1.9744   2.60432  1.85641  1.9954   4.95341  4.0708
  2.57071  1.58166  1.19342  3.78088  1.53254  4.48272  3.98014  2.71951
  1.15942  1.17415  6.41083  3.80554  1.53079  1.05577  7.62581  8.16751
  1.80186  1.91427  1.46729  5.22764  2.11754  1.77624  5.10234  1.32959
  3.03413  2.16991  2.05367  1.30963  4.14644  3.90395  1.84863  1.59201
  1.1192   4.46245  2.40571  5.75911]
35 35


2) *(10 points)* Extend the `make_features` method to also compute quadratic features (`ftype = 'quad'`).

In [54]:
def make_features(X, ftype='quad'):
    """
    generates features from input data, returns Phi.
    ftype is used to distinguish types of features
    """
    n, d = X.shape
    
    if ftype == 'lin': 
        # Linear feature transformation (including intercept)
        #(Rows, Columns)
        #(100 rows, 3 columns)
        Phi = np.empty((n, d+1))
        Phi[:, 0] = 1
        Phi[:, 1:] = X
        
    elif ftype == 'quad':
        # Quadratic feature transformation
        dim = int(1 + (d + (d*(d + 1))/2))
        temp_x_1 = X[:, 0]
        temp_x_2 = X[:, 1]
        x_1_sq = temp_x_1*temp_x_1
        x_1_x_2 = temp_x_1*temp_x_2
        x_2_sq = temp_x_2 * temp_x_2
        feat = np.array([x_1_sq, x_1_x_2, x_2_sq])
        print(dim, len(feat))
        Phi = np.empty((n, dim))
        Phi[:, 0] = 1
        Phi[:, 1:d+1] = X
        Phi[:, d+1:] = feat.T
        
        # Implement your solution here.
    
    else:
        raise Exception(f'Feature type {ftype} not implemented yet')
    
    return Phi
feat = make_features(X_lin)
print(feat)

6 3
[[ 1.00000000e+00  9.74633000e-01 -7.92363000e-01  9.49909485e-01
  -7.72263128e-01  6.27839124e-01]
 [ 1.00000000e+00  3.22351000e-01  1.80340000e+00  1.03910167e-01
   5.81327793e-01  3.25225156e+00]
 [ 1.00000000e+00  1.35589000e+00  3.89593000e-01  1.83843769e+00
   5.28245253e-01  1.51782706e-01]
 [ 1.00000000e+00  8.59725000e-02 -1.06930000e+00  7.39127076e-03
  -9.19303942e-02  1.14340249e+00]
 [ 1.00000000e+00 -6.43600000e-01 -4.00372000e-01  4.14220960e-01
   2.57679419e-01  1.60297738e-01]
 [ 1.00000000e+00 -8.85820000e-01  6.02299000e-01  7.84677072e-01
  -5.33528500e-01  3.62764085e-01]
 [ 1.00000000e+00 -5.72700000e-02 -6.80160000e-01  3.27985290e-03
   3.89527632e-02  4.62617626e-01]
 [ 1.00000000e+00  1.31650000e+00 -5.70268000e-01  1.73317225e+00
  -7.50757822e-01  3.25205592e-01]
 [ 1.00000000e+00 -1.22690000e+00  3.72067000e-02  1.50528361e+00
  -4.56489002e-02  1.38433852e-03]
 [ 1.00000000e+00 -3.07733000e-01 -2.17767000e-01  9.46995993e-02
   6.70140922e-02  4.

3) *(10 points)* Implement Ridge Regression to fit a polynomial function to the data sets with the regularization parameter `lambda_` and feature type `ftype`.

Fill out the methods in `RidgeRegression` to train (`fit`) and predict (`predict`). Feel free to introduce new fields and methods based on your needs, but the methods `fit` and `predict` are required and their interface should not be changed. You need to store the vector of regression coefficients in the field `self.beta`. Before calculating the inverse check if the determinant is non-zero.

In [4]:
class RidgeRegression(object):
    def __init__(self, lambda_, ftype = 'lin'):
        self.lambda_ = lambda_
        self.ftype = ftype
        self.beta = None  # Learned regression coefficients.
    
    def fit(self, X, y):
        """
        X is an array of shape (n, d), 
            where n is the number of samples and d is the number of features.
        y is an array of shape (n,)
        """
        
        Phi = make_features(X, self.ftype)
        
        # Implement your solution here.
    
    def predict(self, X):
        """
        X is an array with shape (n, d).
        The method returns an array of shape (n,).
        """
        Phi = make_features(X, self.ftype)
        
        # Implement your solution here.
        
        return None

4) *(5 points)* Implement the function `MSE` to compute the mean squared error. `y_pred` and `y_true` are the vectors of predicted and true function outputs respectively with shape `(n,)`, where `n` is the number of samples. The function returns a single float.

In [5]:
def MSE(y_pred, y_true):
    """
    return the mean squared error of y_pred and y_true
    """
    # Implement your solution here.
    return None

5) *(30 points)* Evaluate your Ridge Regression model with linear features on the linear `(X_lin, y_lin)` data set. Report the MSE on the full data set when trained on the full dataset. Repeat this for different Ridge regularization parameters `lambda_` and generate a nice plot of the MSE for various `lambda_`. Plot the values of `lambda_` on the x-axis on a logarithmical scale and the error on the y-axis. print the minimal `lambda_`.

How does it perform with quadratic features on this data set?

In [6]:
import matplotlib.pyplot as plt

def train_evaluate(regression_model, X, y):
    """
    Use X and y to fit the regression_model and make prediction over the same dataset.
    Print the error
    """
    regression_model.fit(X, y)
    yhat = regression_model.predict(X)
    print(f'MSE: {MSE(yhat, y)}, feature type: {regression_model.ftype}')
    
def plot_data_and_model(regression_model, X, y):
    """
    Generates a 3D plot of the regression result including the true underlying data.
    The data points are indicated by circles, the prediction is shown as a surface
    """
    fig = plt.figure(figsize=(8, 8))
    plt.suptitle(f'{regression_model.ftype} Features')
    ax = fig.add_subplot(111, projection = '3d')
    ax.scatter(X[:, 0], X[:, 1], y, marker = 'o')
    
    x_min = X.min(0)
    x_max = X.max(0)

    x0_grid, x1_grid = np.mgrid[x_min[0]:x_max[0]:.3, x_min[1]:x_max[1]:.3]

    x_dim_0, x_dim_1 = np.shape(x0_grid)
    x_size = np.size(x0_grid)

    x0_hat = x0_grid.flatten()
    x1_hat = x1_grid.flatten()
    x0_hat = x0_hat.reshape((np.size(x0_hat), 1))
    x1_hat = x1_hat.reshape((np.size(x1_hat), 1))
    x_hat = np.append(x0_hat, x1_hat, 1)
    x_hat_fv = make_features(x_hat, regression_model.ftype)
    y_hat = x_hat_fv.dot(regression_model.beta)
    y_grid = y_hat.reshape((x_dim_0, x_dim_1))
    ax.plot_wireframe(x0_grid, x1_grid, y_grid)
    ax.auto_scale_xyz([x_min[0], x_max[0]], [x_min[1], x_max[1]], [y.min(), y.max()])
    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')
    ax.set_zlabel('$y$')
    
# Implement your solution here.

6) *(5 points)* Evaluate the quadratic dataset `(X_quad, y_quad)` for different values of `lambda_`. Report the MSE on the full data set when trained on the partial dataset `(X_quad_train, y_quad_train)`. Repeat this for different Ridge regularization parameters `lambda_` and generate a nice plot of the MSE for various `lambda_`. Plot the values of `lambda_` on the x-axis on a logarithmical scale and the error on the y-axis. print the minimal `lambda_`.

Plot the surface and data points of the best `lambda_` value using the function `plot_data_and_model`.

In [7]:
# Implement your solution here.

7) *(5 points)* Evaluate the noisy dataset `(X_noise, y_noise)` for different values of `lambda_`. Report the MSE on the full data set when trained on the partial dataset `(X_noise_train, y_noise_train)`. Repeat this for different Ridge regularization parameters `lambda_` and generate a nice plot of the MSE for various `lambda_`. Plot the values of `lambda_` on the x-axis on a logarithmical scale and the error on the y-axis.

Plot the surface and data points of the best `lambda_` value using the function `plot_data_and_model`.

In [8]:
# Implement your solution here.

## Task 3 Evaluation (20 points)

1) *(5 points)* What was the best choice for regularization term `lambda_` in the models above? Explain the observation from the previous task? If `lambda_` is set to zero $\hat{\beta}$ is not regularized, when would $\lambda = 0$ be a good choice?

==> *Briefly explain the observation from the previous task.* (double klick here to edit)

**For all students other than B.Sc. Data Science:** 

2) *(15 points)* Implement the function `cross_validation` for `k_fold = 10` to evaluate the prediction error of your model. Report the mean squared error from cross-validation. Repeat this for different Ridge regularization parameters `lambda_` and generate a nice bar plot of the MSE for various `lambda_`. Plot the values of `lambda_` on the x-axis on a logarithmical scale and the error on the y-axis. 

In [9]:
def cross_validation(regression_model, X, y, k_fold = 10):
    """
    partition data X in k_fold equal sized subsets D = {D_1, ..., D_{k_fold}}, 
        fit the model on k_fold-1 subsets (D\D_i), 
        compute MSE on the evaluatin set (D_i),
        return the mean MSE over all subsets D
    """
    return None